In [18]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten

# https://www.tensorflow.org/api_docs/python/tf/keras/layers/ZeroPadding2D
# ZeroPadding2D This layer can add rows and columns of zeros at the top, bottom, left and right side of an image tensor

from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.datasets import mnist
from keras.utils import np_utils
import keras

# loads the MNIST dataset
(x_train, y_train), (x_test, y_test)  = mnist.load_data()

# Lets store the number of rows and columns
img_rows = x_train[0].shape[0]
img_cols = x_train[1].shape[0]

# Getting our date in the right 'shape' needed for Keras
# We need to add a 4th dimenion to our data thereby changing our
# Our original image shape of (60000,28,28) to (60000,28,28,1)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

# store the shape of a single image 
input_shape = (img_rows, img_cols, 1)

# change our image type to float32 data type
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize our data by changing the range from (0 to 255) to (0 to 1)
x_train /= 255
x_test /= 255

# Now we one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

num_classes = y_test.shape[1]
num_pixels = x_train.shape[1] * x_train.shape[2]

In [19]:
from sklearn.model_selection import GridSearchCV

In [26]:
# create model
def create_model(optimizer):
    model = Sequential()

# 2 sets of CRP (Convolution, RELU, Pooling)
    model.add(Conv2D(20, (5, 5),
                 padding = "same", 
                 input_shape = input_shape))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))

    model.add(Conv2D(50, (5, 5),
                 padding = "same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))

# Fully connected layers (w/ RELU)
    model.add(Flatten())
    model.add(Dense(500))
    model.add(Activation("relu"))

# Softmax (for classification)
    model.add(Dense(num_classes))
    model.add(Activation("softmax"))
           
    model.compile(loss = 'categorical_crossentropy',
              optimizer = keras.optimizers.Adadelta(),
              metrics = ['accuracy'])
    
    print(model.summary())


In [28]:
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import to_categorical
model = KerasClassifier(build_fn=create_model, verbose=0)
parameters = {'batch_size': [25, 32],
              'epochs': [100, 500],
              'optimizer': ['adam', 'rmsprop']}

grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10, 
                           n_jobs=-1)


grid_search = grid_search.fit(x_train, y_train)

AttributeError: 'NoneType' object has no attribute 'loss'

In [23]:
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(x_train, y_train)

AttributeError: 'NoneType' object has no attribute 'loss'